https://www.youtube.com/watch?v=tcqEUSNCn8I
https://github.com/pixegami/langchain-rag-tutorial/blob/main/query_data.py

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
#from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
#from dotenv import load_dotenv
import os
import shutil
from langchain_community.llms import Ollama
import json

In [5]:
paper_id = "2402.01383v1"
#CHROMA_PATH = paper_id+"/chroma"
CHROMA_PATH = paper_id+"/chroma300"
DATA_PATH = paper_id

In [44]:

def store_summary(survey1,filename):
    with open(f'summaries/'+filename+'.txt', 'w') as file:
        file.write(survey1)
def write_json(documents_data,file_name):
    converted_data = [dict(item) for item in documents_data['source_documents']]
    dict_data = {'answer':documents_data['result'],'source':converted_data}
    with open('summaries/'+file_name+'.json', 'w') as file:
        json.dump(dict_data, file, indent=4)


In [4]:

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
    documents = loader.load()
    return documents

In [5]:
documents = load_documents()

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
documents[4].metadata

{'source': '2402.01383v1/2307.07889v3.LLM_Comparative_Assessment__Zero_shot_NLG_Evaluation_through_Pairwise_Comparisons_using_Large_Language_Models.pdf'}

In [7]:
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [8]:
chunks = split_text(documents)

Split 45 documents into 20320 chunks.
language processing benchmarks. In response to this challenge, recent studies commonly introduce an evaluation approach, namely the Elo rating sys- tem (Elo, 1967), wherein either human or LLM judges are enlisted to adjudicate between two LLM- generated outputs (Askell et al., 2021; Bai et al.,
{'source': '2402.01383v1/2307.03025v3.Style_Over_Substance__Evaluation_Biases_for_Large_Language_Models.pdf', 'start_index': 2067}


In [9]:
#chunks[11].page_content
some = chunks[0:10]

In [7]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    #if os.path.exists(CHROMA_PATH):
     #   shutil.rmtree(CHROMA_PATH)


    # Ensure the directory exists and has the correct permissions
    os.makedirs(CHROMA_PATH, exist_ok=True)
   # os.chmod(CHROMA_PATH, 0o755)
        
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


In [12]:
save_to_chroma(chunks)

Saved 20320 chunks to 2402.01383v1/chroma300.


/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [8]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [14]:
from langchain.chains import RetrievalQA

In [15]:
llm = Ollama(model = "mistral",temperature=0)
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
from langchain.chains import RetrievalQA
rag = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=db.as_retriever(),
            return_source_documents = True,
            
            #memory=ConversationSummaryMemory(llm = Ollama(model="mistral")),
            #chain_type_kwargs={"prompt": pt, "verbose": True},
        )

answer = rag.invoke("What is the recent development of LLM generated text evaluation?")
rag_overview = answer['result']

In [16]:
print(rag_overview)
store_summary(rag_overview,'rag_overview300')

 Recent developments in LLM (Large Language Model) generated text evaluation include using LLMs to evaluate the quality of their own text outputs as an alternative to human evaluations. This approach can provide a more comprehensive understanding of LLM performance and predict how a tuned version might behave in various scenarios. Additionally, applying LLMs as evaluators for text generation has been shown to be scalable, cost-effective, and can significantly enhance the ability to assess the quality of generated text. For instance, Fu et al. (2023) use LLM's predicted text probability as the automated score. Furthermore, this approach has been applied to evaluate Chinese LLMs, resulting in a faster evaluation process and decreased average annotation cost per sample. However, it is important to note that the utilization of LLMs as evaluators for text generation is still in the exploratory phase, and there are limitations that provide opportunities for future work, such as the performan

In [43]:
a = answer['source_documents'][0]
converted_data = [dict(item) for item in answer['source_documents']]
#dict(answer['source_documents'])
converted_data
answer

{'query': 'What is the recent development of LLM generated text evaluation?',
 'result': " Recent developments in LLM (Large Language Model) generated text evaluation include using LLMs to evaluate the quality of their own text outputs as an alternative to human evaluations. This approach can provide a more comprehensive understanding of LLM performance and predict how a tuned version might behave in various scenarios. Additionally, applying LLMs as evaluators for text generation has been shown to be scalable, cost-effective, and can significantly enhance the ability to assess the quality of generated text. For instance, Fu et al. (2023) use LLM's predicted text probability as the automated score. Furthermore, this approach has been applied to evaluate Chinese LLMs, resulting in a faster evaluation process and decreased average annotation cost per sample. However, it is important to note that the utilization of LLMs as evaluators for text generation is still in the exploratory phase, a

In [45]:
write_json(answer,'rag_overview300')

In [48]:
type(answer)
sources = [doc.metadata.get("source", None) for doc, _score in answer]
sources

ValueError: too many values to unpack (expected 2)

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


retriever=db.as_retriever()
llm = Ollama(model = "mistral",temperature=0)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)


In [11]:
query = 'What is the recent development of LLM generated text evaluation?'
chain.invoke({"input": query})

{'input': 'What is the recent development of LLM generated text evaluation?',
 'context': [],
 'answer': " Recent developments in LLM (Large Language Model) generated text evaluation include the use of specialized metrics like BLEU, ROUGE, METEOR, and PERSIAN to assess the quality and relevance of text generated by these models. Additionally, there is ongoing research into developing new evaluation methods that can better capture the nuances and complexities of human language, such as those based on contextual understanding and common sense reasoning. However, it's important to note that no single metric or method can perfectly evaluate the performance of LLMs in all situations."}

TypeError: 'VectorStoreRetriever' object is not callable